## ВМА. Лабораторная 6. Отчет
### Гамезо Валерия

### Постановка задачи

Разработать программу вычисления наибольшего и второго по величине модуля собственных значений и соответствующих им собственных векторов симметричной матрицы. 

### Расчетные формулы

Рассчет $\lambda_1$ для несимметричной матрицы: 

$$\lambda_1 \approx \nu_i^{k+1} \; \text{sign} \; u_i^k, \text{где} \; i = \overline{1, n}, \nu_i^k = \| \nu^k \|_{\infty} $$

Для симметричной матрицы: 

$$\lambda_1 \approx \frac{(\nu^{k + 1}, u^k)}{(u^k, u^k)}$$

Рассчет вектора "погрешности" для $\lambda_1$:

$$ \nu^{k+1} – \lambda_1 u^k  $$

Рассчет $\lambda_2$:

$$ \lambda_2 \approx \frac{\nu^{m + 1}_i \| \nu^m \| - \lambda_1 \nu_i^m}{\nu_i^m - \lambda_1 u_i^{m - 1}}, \text{где} \; i = \overline{1, n}, \nu_i^m - \lambda_1 u_i^{m - 1} = \| \nu^m - \lambda_1 u^{m - 1} \|_{\infty}$$

Рассчет собственного вектора для $\lambda_2$:

$$ \nu^{k + 1} - \lambda_1 u^k $$

Рассчет вектора "погрешности" для $\lambda_2$:

$$ A x – \lambda_2 x  $$

### Входные данные

$$
A = \left( \begin{array}{rrrrrrrrrrrr}
  29 &  -3 &  -4 &  -2 &   0 &  -1 &  -1 &  -3 &  -3 &  -4 &  -4 &  -3 & \\
  -3 &  28 &  -4 &  -3 &  -3 &  -1 &  -3 &  -2 &  -3 &  -1 &  -1 &  -4 & \\
  -4 &  -4 &  24 &   0 &  -3 &   0 &  -4 &  -2 &  -2 &  -1 &  -1 &  -3 & \\
  -2 &  -3 &   0 &  15 &  -1 &   0 &   0 &  -2 &  -1 &  -2 &  -1 &  -3 & \\
   0 &  -3 &  -3 &  -1 &  27 &  -4 &  -1 &  -3 &  -1 &  -4 &  -4 &  -3 & \\
  -1 &  -1 &   0 &   0 &  -4 &  13 &  -1 &  -3 &   0 &  -3 &   0 &   0 & \\
  -1 &  -3 &  -4 &   0 &  -1 &  -1 &  20 &  -1 &  -3 &  -4 &  -1 &  -1 & \\
  -3 &  -2 &  -2 &  -2 &  -3 &  -3 &  -1 &  25 &   0 &  -4 &  -3 &  -2 & \\
  -3 &  -3 &  -2 &  -1 &  -1 &   0 &  -3 &   0 &  19 &  -1 &  -2 &  -3 & \\
  -4 &  -1 &  -1 &  -2 &  -4 &  -3 &  -4 &  -4 &  -1 &  29 &  -2 &  -3 & \\
  -4 &  -1 &  -1 &  -1 &  -4 &   0 &  -1 &  -3 &  -2 &  -2 &  22 &  -3 & \\
  -3 &  -4 &  -3 &  -3 &  -3 &   0 &  -1 &  -2 &  -3 &  -3 &  -3 &  28 &
\end{array} \right)
$$

### Листинг программы

Класс матрицы

```c++
class Matrix {
public:
    Matrix(const Matrix& matrix) : mSize(matrix.mSize) { //конструктор копирования
        a = new float* [mSize];

        for (int i = 0; i < mSize; ++i) {
            a[i] = new float[mSize];

            for (int j = 0; j < mSize; ++j) {
                a[i][j] = matrix[i][j];
            }
        }
    }
    
    Matrix(int size) : mSize(size) {
        a = new float* [mSize];

        for (int i = 0; i < mSize; ++i) {
            a[i] = new float[mSize];
        }

        generateCoefs();
    }

    ~Matrix() {
        for (int i = 0; i < mSize; ++i) {
            delete[] a[i];
        }
    }


    float* operator[] (int index) const {
        return a[index];
    }
    
    int size() const {
        return mSize;
    }
    
    Matrix& operator= (const Matrix& other) {
        if (this == &other) return *this;

        for (int i = 0; i < mSize; ++i) {
            for (int j = 0; j < mSize; ++j) {
                a[i][j] = other[i][j];
            }
        }

        return *this;
    }

private:
    void generateCoefs (bool isNull) { // генерация коеффициентов симметричной матрицы
    srand(time(nullptr));

    for (int i = 0; i < mSize; ++i) {
        for (int j = i + 1; j < mSize; ++j) {
            a[i][j] = std::rand() % 5 - 4;
            a[j][i] = a[i][j];
        }
    }
    
    for (int i = 0; i < mSize; ++i) {
        for (int j = 0; j < mSize; ++j) {
            if (i != j) {
                a[i][i] -= a[i][j];
            }
        }
    }
    
    a[0][0] ++;
    };
```

Класс, выполняющий итерации степенного метода и вычисляющий собственные вектора двух наибольших по модулю собственных значений заданной матрицы. Файл PowerMethod.h

```c++
    
class PowerMethod {
    public:
        explicit PowerMethod(const Matrix&);
        
        std::vector<float> getV();
        std::vector<float> getU();

        void calcNextIt();

        float getFirstLambdaCurEigenVal(bool); // вычисляет наибольшее по модулю собственное значение lambda_1
                                            // для симместричной и нессиметричной матриц, 
                                            // в зависимости от установленного логического флага
        std::vector<float> getFirstLambdaErrorVector(); // вычисляет значение вектора погрешности для 
                                                        // lambda_1 на текущей итерации k
    
        float getSecondLambdaEigenVal(std::vector<float>, std::vector<float>, std::vector<float>, float); // вычисляет
                                            // второе наибольшее по модулю собственное значение lambda_2
                                            // при заданных векторах v^{m + 1}, v^m, u^{m - 1} 
                                            // и значении lambda_1
        std::vector<float> getSecondLambdaEigenVector(std::vector<float>, std::vector<float>, float); // вычисляет
                                            // собственный вектор для lambda_2
                                            // при заданных векторах v^{m + 1}, u^m 
                                            // и значении lambda_1
        std::vector<float> getSecondLambdaErrorVector(std::vector<float>, float); // вычисляет значение 
                                                                    // вектора погрешности (собственный вектор x 
                                                                    // и собственное значение lambda_2 
                                                                    // заданы аргументами)

    private:
        Matrix matrix;
        std::vector<float> u; // нормированный вектор u_k при k, равном значении curI
        std::vector<float> v; // вектор v_{k + 1} при k, равном значении curI
        int curI;
};
```

PowerMethod.cpp

```c++
PowerMethod::PowerMethod(const Matrix& otherMatrix) // инициализация начального состояния
                                                    // u = u_0, k = 0
    : matrix(otherMatrix), curI(-1) {
  int size = matrix.size();

  u.resize(size);
  v.resize(size);
  v[0] = 1;

  calcNextIt();
}

std::vector<float> PowerMethod::getV() {
  return v;
}

std::vector<float> PowerMethod::getU() {
  return u;
}

void PowerMethod::calcNextIt() { // вычисление следующей итерации
  int size = matrix.size();

  float maxV = v[0]; // нормировка v^k для получения u^k
  for (int i = 0; i < size; ++i) {
    if (fabs(v[i]) > fabs(maxV)) {
      maxV = fabs(v[i]);
    }

    u[i] = v[i];
  } 

  for (int i = 0; i < size; ++i) {
    u[i] /= maxV;
  }

  for (int i = 0; i < size; ++i) { // вычисление v^{k + 1}
    v[i] = 0;

    for (int j = 0; j < size; ++j) {
      v[i] += matrix[i][j] * u[j];
    }
  }

  curI++; // инкремент счетчика
}

float PowerMethod::getFirstLambdaCurEigenVal(bool isSymmetric) {
  float lambda;
  int size = matrix.size();

  if (isSymmetric) { // вычисление lambda_1 для симметричной матрицы
    float num, den;
    num = den = 0;

    for (int i = 0; i < size; ++i) {
      num += v[i] * u[i]; // вычисление векторного произведения (v^{k + 1}, u^k)
      den += u[i] * u[i]; // вычисление векторного произведения (u^k, u^k)
    }

    lambda = num / den;
  } else { // вычисление lambda_1 для несимметричной матрицы
    int maxUIndex = 0;

    for (int i = 1; i < size; ++i) { // нахождение максиальной нормы вектора v^{k + 1}
      if (fabs(u[i]) > fabs(u[maxUIndex])) {
        maxUIndex = i;
      }
    }

    lambda = u[maxUIndex] < 0 ? -v[maxUIndex] : v[maxUIndex]; // умножение lambda_1 на sign (u^k_i)
  }

  return lambda;
}

std::vector<float> PowerMethod::getFirstLambdaErrorVector() {
  int size = matrix.size();
  float lambda = getFirstLambdaCurEigenVal(true);

  std::vector<float> error(size);

  for (int i = 0; i < size; ++i) {
    error[i] = v[i] - lambda * u[i]; // вычисление вектора v^{k + 1} - lambda_1 * u^k
  }

  return error;
}

float PowerMethod::getSecondLambdaEigenVal(std::vector<float> u_mMinus,
                                           std::vector<float> v_m,
                                           std::vector<float> v_mPlus,
                                           float lambda1) {
  int maxAbsDenIndex = 0;
  int size = matrix.size();
  float v_mNorm = fabs(v_m[0]);

  for (int i = 1; i < size; ++i) {
    if (v_mNorm < fabs(v_m[i])) {
      v_mNorm = fabs(v_m[i]); // вычисление нормы вектора v^m
    }
  }

  for (int i = 1; i < size; ++i) {
    if (fabs(v_m[maxAbsDenIndex] - lambda1 * u_mMinus[maxAbsDenIndex]) <
        fabs(v_m[i] - lambda1 * u_mMinus[i])) {
      maxAbsDenIndex = i; // нахождение индекса i, для которого v^m_i - lambda_1 * u^{m - 1}_i 
                            // равно максимум норме вектора v^m - lambda_1 * u^{m - 1} 
    }
  }

  return (v_mPlus[maxAbsDenIndex] * v_mNorm - lambda1 * v_m[maxAbsDenIndex])
      / (v_m[maxAbsDenIndex] - lambda1 * u_mMinus[maxAbsDenIndex]);
}

std::vector<float> PowerMethod::getSecondLambdaEigenVector(std::vector<float> v_mPlus,
                                               std::vector<float> u_m,
                                               float lambda1) {
  int size = matrix.size();
  std::vector<float> x(size);

  for (int i = 0; i < size; ++i) {
    x[i] = v_mPlus[i] - lambda1 * u_m[i];
  }

  return x;
}

std::vector<float> PowerMethod::getSecondLambdaErrorVector(std::vector<float> x, float lambda2) {
  int size = matrix.size();

  std::vector<float> error(size);

  for (int i = 0; i < size; ++i) {
    error[i] = 0;

    for (int j = 0; j < size; ++j) {
      error[i] = matrix[i][j] * x[j];
    }

    error[i] -= lambda2 * x[i];
  }

  return error;
}
```

main.cpp

```c++
const int N = 12;

int main() {
  Matrix m(N);

  std::vector<float> u_m30Minus; // значение вектора u^{m - 1} для m = 30 для вычисления lambda_1
  std::vector<float> v_m30; // значение вектора v^m для m = 30 для вычисления lambda_1 
  std::vector<float> v_m30Plus; // значение вектора v^{m + 1} для m = 30 для вычисления lambda_1 

  std::vector<float> u_m50Minus; // значение вектора u^{m - 1} для m = 50 для вычисления lambda_1 
  std::vector<float> v_m50; // значение вектора v^m для m = 50 для вычисления lambda_1 
  std::vector<float> v_m50Plus; // значение вектора v^{m + 1} для m = 50 для вычисления lambda_1 

  std::vector<float> u_k; // значение вектора u^k при k = 50 
  std::vector<float> v_kPlus; // значение вектора v^{k + 1} при k = 50 

  PrintToFile::printMatrix(m, "matrix"); // вывод матрицы в файл "matrix.txt"

  PowerMethod powerMethod(m); 
  
  for (int i = 0; i < 46; ++i) { // первые 45 итераций метода
    if (i == 30) {
      u_m30Minus = powerMethod.getU();
      v_m30 = powerMethod.getV();

      powerMethod.calcNextIt();

      v_m30Plus = powerMethod.getV();
    }

    powerMethod.calcNextIt();
  }

  std::vector<float> errorVector;
  float error;
  for (int i = 46; i < 51; ++i) { // итерации 46 - 50 с выводом собственного значения lambda_1, собственного вектора,
                                  // отвечающего значению lambda_1, вектора погрешности и максимум нормы
                                  // вектора погрешности в отдельный файл
    if (i == 50) {
      u_m50Minus = powerMethod.getU();
      v_m50 = powerMethod.getV();
    }

    powerMethod.calcNextIt();

    errorVector = powerMethod.getFirstLambdaErrorVector();
    error = fabs(errorVector[0]);
    for (int i = 1; i < N; ++i) {
      if (fabs(error) < fabs(errorVector[i])) {
        error = fabs(errorVector[i]);
      }
    }

    PrintToFile::printEigen(powerMethod.getFirstLambdaCurEigenVal(true),
                            powerMethod.getU(),
                            errorVector,
                            error,
                            "lambda" + std::to_string(i));
  }

  u_k = powerMethod.getU();
  v_m50Plus = powerMethod.getV();
  v_kPlus = powerMethod.getV();

  float lambda1, lambda2, lambda2Error;
  std::vector<float> x;
  std::vector<float> lambda2ErrorVector;

  // вычисление lambda_2 и сопутствующих значений при m = 30 и 
  // при вычислении lambda_1 при помощи формулы для несимметричнх матриц

  lambda1 = powerMethod.getFirstLambdaCurEigenVal(false);
  lambda2 = powerMethod
      .getSecondLambdaEigenVal(u_m30Minus, v_m30, v_m30Plus, lambda1);
  x = powerMethod.getSecondLambdaEigenVector(v_kPlus, u_k, lambda1);
  lambda2ErrorVector = powerMethod.getSecondLambdaErrorVector(x, lambda2);

  lambda2Error = fabs(lambda2ErrorVector[0]);
  for (int i = 1; i < N; ++i) {
    if (fabs(lambda2Error) < fabs(lambda2ErrorVector[i])) {
      lambda2Error = fabs(lambda2ErrorVector[i]);
    }
  }

  PrintToFile::printEigen(lambda2, x, lambda2ErrorVector,
                          lambda2Error, "lambda2FirstCase");

  // вычисление lambda_2 и сопутствующих значений при m = 50 и 
  // при вычислении lambda_1 при помощи формулы для несимметричнх матриц

  lambda2 = powerMethod
      .getSecondLambdaEigenVal(u_m50Minus, v_m50, v_m50Plus, lambda1);
  lambda2ErrorVector = powerMethod.getSecondLambdaErrorVector(x, lambda2);

  lambda2Error = fabs(lambda2ErrorVector[0]);
  for (int i = 1; i < N; ++i) {
    if (fabs(lambda2Error) < fabs(lambda2ErrorVector[i])) {
      lambda2Error = fabs(lambda2ErrorVector[i]);
    }
  }

  PrintToFile::printEigen(lambda2, x, lambda2ErrorVector,
                          lambda2Error, "lambda2SecondCase");

  // вычисление lambda_2 и сопутствующих значений при m = 50 и 
  // при вычислении lambda_1 при помощи формулы для симметричнх матриц

  lambda1 = powerMethod.getFirstLambdaCurEigenVal(true);
  lambda2 = powerMethod
      .getSecondLambdaEigenVal(u_m50Minus, v_m50, v_m50Plus, lambda1);
  x = powerMethod.getSecondLambdaEigenVector(v_kPlus, u_k, lambda1);
  lambda2ErrorVector = powerMethod.getSecondLambdaErrorVector(x, lambda2);

  lambda2Error = fabs(lambda2ErrorVector[0]);
  for (int i = 1; i < N; ++i) {
    if (fabs(lambda2Error) < fabs(lambda2ErrorVector[i])) {
      lambda2Error = fabs(lambda2ErrorVector[i]);
    }
  }

  PrintToFile::printEigen(lambda2, x, lambda2ErrorVector,
                          lambda2Error, "lambda2ThirdCase");

  return 0;
}
```

### Выходные данные

$$
\begin{array}{cccccc}
    k & 46 & 47 & 48 & 49 & 50 \\
    \lambda_1 & 36.5011864 & 36.5011978 & 36.5012054 & 36.5012093 & 36.5012169 \\
    X & \left( \begin{array}{r}
    1.0000000 \\
    -0.4650617 \\
    -0.3935440 \\
    0.0172827 \\
    0.7475476 \\
    -0.0481157 \\
    0.3375207 \\
    0.0786399 \\
    -0.0818869 \\
    -0.9899890 \\
    -0.3525443 \\
    0.1775544 \\
\end{array} \right) & \left( \begin{array}{r}
    1.0000000 \\
    -0.4654585 \\
    -0.3935365 \\
    0.0173006 \\
    0.7475678 \\
    -0.0480674 \\
    0.3376659 \\
    0.0787914 \\
    -0.0818949 \\
    -0.9903872 \\
    -0.3525973 \\
    0.1780188 \\
\end{array} \right) & \left( \begin{array}{r}
    1.0000000 \\
    -0.4658148 \\
    -0.3935296 \\
    0.0173166 \\
    0.7475856 \\
    -0.0480241 \\
    0.3377959 \\
    0.0789266 \\
    -0.0819021 \\
    -0.9907433 \\
    -0.3526449 \\
    0.1784359 \\
\end{array} \right) & \left( \begin{array}{r}
    1.0000000 \\
    -0.4661346 \\
    -0.3935231 \\
    0.0173309 \\
    0.7476010 \\
    -0.0479852 \\
    0.3379123 \\
    0.0790473 \\
    -0.0819087 \\
    -0.9910616 \\
    -0.3526875 \\
    0.1788105 \\
\end{array} \right) & \left( \begin{array}{r}
    1.0000000 \\
    -0.4664217 \\
    -0.3935170 \\
    0.0173437 \\
    0.7476143 \\
    -0.0479502 \\
    0.3380164 \\
    0.0791549 \\
    -0.0819146 \\
    -0.9913458 \\
    -0.3527258 \\
    0.1791468 \\
\end{array} \right) \\ 
\nu^{k+1} – \lambda_1 u^k & \left( \begin{array}{r}
    1.0000000 \\
    -0.4650617 \\
    -0.3935440 \\
    0.0172827 \\
    0.7475476 \\
    -0.0481157 \\
    0.3375207 \\
    0.0786399 \\
    -0.0818869 \\
    -0.9899890 \\
    -0.3525443 \\
    0.1775544 \\
\end{array} \right) & \left( \begin{array}{r}
    1.0000000 \\
    -0.4654585 \\
    -0.3935365 \\
    0.0173006 \\
    0.7475678 \\
    -0.0480674 \\
    0.3376659 \\
    0.0787914 \\
    -0.0818949 \\
    -0.9903872 \\
    -0.3525973 \\
    0.1780188 \\
\end{array} \right) & \left( \begin{array}{r}
    1.0000000 \\
    -0.4658148 \\
    -0.3935296 \\
    0.0173166 \\
    0.7475856 \\
    -0.0480241 \\
    0.3377959 \\
    0.0789266 \\
    -0.0819021 \\
    -0.9907433 \\
    -0.3526449 \\
    0.1784359 \\
\end{array} \right) & \left( \begin{array}{r}
    1.0000000 \\
    -0.4661346 \\
    -0.3935231 \\
    0.0173309 \\
    0.7476010 \\
    -0.0479852 \\
    0.3379123 \\
    0.0790473 \\
    -0.0819087 \\
    -0.9910616 \\
    -0.3526875 \\
    0.1788105 \\
\end{array} \right) & \left( \begin{array}{r}
    1.0000000 \\
    -0.4664217 \\
    -0.3935170 \\
    0.0173437 \\
    0.7476143 \\
    -0.0479502 \\
    0.3380164 \\
    0.0791549 \\
    -0.0819146 \\
    -0.9913458 \\
    -0.3527258 \\
    0.1791468 \\
\end{array} \right) \\ 
\| \nu^{k+1} – \lambda_1 u^k \|_{\infty} & 0.0154252 & 0.0138540 & 0.0124421 & 0.0111728 & 0.0100303
\end{array} 
$$

$$
\begin{array}{cccc}
    m & 30 & 50 & 50 \\
    \small \text{formulae for} & \text{not symmetric} & \text{not symmetric} & \text{symmetric} \\
    \lambda_2 & 32.9747467 & 33.0744057 & 32.7662621 \\ 
    X &
    \left( \begin{array}{r}
        0.0000000 \\
        -0.0094051 \\
        0.0002003 \\
        0.0004171 \\
        0.0004292 \\
        0.0011443 \\
        0.0034018 \\
        0.0035067 \\
        -0.0001974 \\
        -0.0092812 \\
        -0.0012550 \\
        0.0110178 \\
    \end{array} \right) &
    \left( \begin{array}{r}
        0.0000000 \\
        -0.0094051 \\
        0.0002003 \\
        0.0004171 \\
        0.0004292 \\
        0.0011443 \\
        0.0034018 \\
        0.0035067 \\
        -0.0001974 \\
        -0.0092812 \\
        -0.0012550 \\
        0.0110178 \\
    \end{array} \right) &
    \left( \begin{array}{r}
    -0.0055122 \\
    -0.0068340 \\
    0.0023699 \\
    0.0003215 \\
    -0.0036907 \\
    0.0014086 \\
    0.0015383 \\
    0.0030704 \\
    0.0002542 \\
    -0.0038147 \\
    0.0006895 \\
0.0100303 \\
    \end{array} \right) \\
    A x – \lambda_2 x &
    \left( \begin{array}{r}
       -0.0330534 \\
        0.2660608 \\
        -0.0396573 \\
        -0.0468076 \\
        -0.0472046 \\
        -0.0377327 \\
        -0.1231899 \\
        -0.1376668 \\
        -0.0265438 \\
        0.2729905 \\
        0.0083311 \\
        -0.0548108 \\
    \end{array} \right) &
    \left( \begin{array}{r}
        -0.0330534 \\
        0.2669981 \\
        -0.0396773 \\
        -0.0468492 \\
        -0.0472474 \\
        -0.0378467 \\
        -0.1235289 \\
        -0.1380163 \\
        -0.0265242 \\
        0.2739154 \\
        0.0084562 \\
        -0.0559088 \\
    \end{array} \right) &
    \left( \begin{array}{r}
        0.1505246 \\
        0.1838045 \\
        -0.1077429 \\
        -0.0406254 \\
        0.0908403 \\
        -0.0461538 \\
        -0.0604338 \\
        -0.1206646 \\
        -0.0384185 \\
        0.0949026 \\
        -0.0526834 \\
        -0.0478069 \\
    \end{array} \right) \\
    \| A x – \lambda_2 x \|_{\infty} & 0.2729905 & 0.2739154 & 0.1838045 
\end{array}
$$

### Выводы

С увеличением числа итераций увеличивается точность вычисления соственных значений матрицы. Собстчвенные значения сходятся быстрее на симметричных матрицах при применеии формул для симметричных матриц.